<a href="https://colab.research.google.com/github/jh6695/study/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_2_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 정수 인코딩

각 단어를 정수에 맵핑 하는 전처리 작업

주로 빈도수가 높은 순서대로 정렬하고 차례대로 숫자 매핑

# 1) 딕셔너리 사용하기

In [124]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [125]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."


In [126]:
# 문장 단위로 토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [127]:
# 정제 정규화 토근화
vocab = {}
preprocessed_sentences = []
stop_word = set(stopwords.words('english'))

for sentence in sentences:
  # 단어 토근화
  tokenize_sentence = word_tokenize(sentence)
  result = []

  for word in tokenize_sentence:
    word = word.lower() # 단어 소문자화
    if word not in stop_word: #불용어 제거
      if len(word) > 2: # 단어수 2개 이하 제거
        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1
  preprocessed_sentences.append(result)
print(preprocessed_sentences)


[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [128]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [129]:
print(vocab['barber'])

8


In [130]:
# 빈도수가 높은 순서대로 정리
vocab_sorted = sorted(vocab.items(), key = lambda x: x[1], reverse= True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [131]:
# 높은 빈도수를 가진 단어부터 정수 부여. 1부터 시작
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
  if frequency > 1 : # 빈도수가 너무 적은 단어 제거
    i = i + 1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [132]:
# 빈도수 상위 n개인 단어만 사용하고 싶은 경우
# n = 5일 때
vocab_size = 5

#인덱스가 5 초과인 단어 제거
word_frequency = [word for word, index in word_to_index.items() if index >= vocab_size + 1]

#해당 단어에 대한 인덱스 정보 삭제
for w in word_frequency:
  del word_to_index[w]

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


word_to_index를 사용해 단어 토큰화가 되어있는 senetneces에 있는 단어를 정수로 바꾸는 작업

단어 집합에 존재하지 않는 단어: Out_Of_Vocablary(OOV)

OOV라는 단어를 추하고 없는 단어에 OOV 인덱스로 인코딩

['barber', 'good', 'person'] -> [1, OOV, 5]

In [133]:
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [134]:
#sentences의 모든 단어를 매핑되는 정수로 인코딩
encoded_sentences = []
for sentence in preprocessed_sentences:
  encoded_sentence = []
  for word in sentence:
    try :
      encoded_sentence.append(word_to_index[word]) # 단어 집합에 있는 단어라면 해당 단어의 정수 리턴
    except:
      encoded_sentence.append(word_to_index['OOV']) # 단어 집합에 없는 단어면 OOV의 정수 리턴
  encoded_sentences.append(encoded_sentence)

print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


# 2) counter 사용

In [135]:
from collections import Counter

In [136]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [137]:
# 단어 집합을 만들기 위해 [,] 제거
# np.hstack으로도 가능. 수평 결합
all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [138]:
# counter(): 중복 제거하고 빈도수 기록
vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [139]:
print(vocab['barber'])

8


In [140]:
# most_common(): 상위 빈도수를 가진 단어만 추출
# 상위 5개 단어만 집합으로 저장
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [141]:
#내림차순 정수 인덱스 부여
word_to_index = {}
i = 0

for (word, frequency) in vocab:
  i = i + 1
  word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


# 3) nltk의 FreqDist 사용하기

In [142]:
from nltk import FreqDist
import numpy as np

In [143]:
vocab = FreqDist(np.hstack(preprocessed_sentences))

In [144]:
print(vocab['barber'])

8


In [145]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [146]:
# enumerate()로 내림차순 정수 부여
word_to_index = {word[0]: index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


# 4) enumerate() 이해하기

순서가 있는 자료형을 입력받아 인덱스를 순차적으로 함께 리턴함

In [147]:
test_input = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test_input):
  print("value: {}, index: {}".format(value, index) )

value: a, index: 0
value: b, index: 1
value: c, index: 2
value: d, index: 3
value: e, index: 4


# 2. 케라스의 텍스트 전처리


In [148]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [149]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [150]:
#fit_on_texts() 빈도수 내림차수 정수 부여
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [151]:
#확인하기 - word_index
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [152]:
#각 단어 카운트 - word_counts
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [153]:
#입력으로 들어온 코퍼스에 대해 각 단어를 이미 정해진 인덱스로 변환 - texts_to_sequences()
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [154]:
#빈도수 상위 5개 추출
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용. 0부터 세기 때문에 +1
tokenizer.fit_on_texts(preprocessed_sentences)

In [155]:
print(tokenizer.word_index)
#적용 안됨

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [156]:
#실제 적용
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [157]:
#word_index와 word_counts에도 상위 5개 만큼의 단어만 남기고 싶다면
vocab_size = 5
word_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size + 1]

#인덱스가 5 초과인 단어 제거
for word in word_frequency:
  del tokenizer.word_index[word]
  del tokenizer.word_counts[word]

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [158]:
#OOV 추가
vocab_size = 5
tokenizer = Tokenizer(num_words= vocab_size + 2, oov_token ='OOV') #0부터 시작함과 OOV를 고려해서 +2
tokenizer.fit_on_texts(preprocessed_sentences)

In [159]:
#기본적으로 OOV의 인덱스는 1로 지정됨
print(tokenizer.word_index['OOV'])

1


In [160]:
#코퍼스에 대해 정수 인코딩
print(tokenizer.texts_to_sequences(preprocessed_sentences))
# OOV - 1, 빈도수 상위 5 - 2~6

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]




---
# 1. 패딩

병렬 연산을 위해 문장들의 길이를 임의로 동일하게 맞추는 작업


In [161]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [162]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [163]:
#단어 집합 만들고 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [164]:
#동일한 길이로 맞추기 위해 가장 긴 문장 길이 계산
max_len = max(len(item) for item in encoded)
print(max_len)

7


제로 패딩

모든 문장의 길이를 7로 맞추기 위해 7보다 짧은 문장의 인덱스에 0을 넣어서 7로 맞춘다.

0은 아무런 의미 없음.

In [165]:
for sentence in encoded:
  while len(sentence) < max_len:
    sentence.append(0)

padded_np = np.array(encoded)
print(padded_np)

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


# 2. 케라스 전처리 도구로 패딩하기

In [169]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [170]:
eocoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

[[1, 5, 0, 0, 0, 0, 0], [1, 8, 5, 0, 0, 0, 0], [1, 3, 5, 0, 0, 0, 0], [9, 2, 0, 0, 0, 0, 0], [2, 4, 3, 2, 0, 0, 0], [3, 2, 0, 0, 0, 0, 0], [1, 4, 6, 0, 0, 0, 0], [1, 4, 6, 0, 0, 0, 0], [1, 4, 2, 0, 0, 0, 0], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13, 0, 0, 0]]


In [172]:
padded = pad_sequences(encoded)
print(padded)

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


In [174]:
# 결과가 동일한지 확인
(padded == padded_np).all()

True

데이터의 길이를 지정해서 맞추는 경우

In [175]:
padded = pad_sequences(encoded, maxlen = 5)
padded # 앞부터 삭제

array([[ 0,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 3,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 6,  0,  0,  0,  0],
       [ 6,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 3, 13,  0,  0,  0]], dtype=int32)

In [178]:
padded = pad_sequences(encoded, maxlen=5, truncating = 'post')
padded #뒤가 삭제

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 7,  7,  3,  2, 10],
       [ 1, 12,  3, 13,  0]], dtype=int32)

In [187]:
#0이 아닌 다른 숫자로 패딩
#현재 있는 최대 숫자의 +1
last_value = len(tokenizer.word_index) + 1
print(last_value)

14


In [188]:
padded = pad_sequences(encoded, padding = 'post',  value = last_value)
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]], dtype=int32)



---

#원-핫 인코딩

단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1을 부여하고, 다른 단어에 0을 부여하는 단어의 벡터 표현 방식. 이렇게 표현된 벡터를 원-핫 벡터라고 한다.

처리 과정

1. 정수 인코딩

2. 표현하고 싶은 단어의 정수를 인덱스로 간수하고 해당 위치에 1 부여. 다른 모든 단어의 인덱스 위치에는 0 부여.

In [193]:
from konlpy.tag import Okt
okt = Okt()
tokens = okt.morphs("나는 자연어 처리를 배운다")
print(tokens)

['나', '는', '자연어', '처리', '를', '배운다']


In [194]:
#토큰에 고유 정수 부여
word_to_index = {word: index for index, word in enumerate(tokens)}
print(word_to_index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [195]:
#원-핫 벡터 만드는 함수 생성
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

In [197]:
one_hot_encoding("자연어", word_to_index)

[0, 0, 1, 0, 0, 0]

# 2. 케라스를 이용한 원-핫 인코딩

In [198]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

In [200]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [202]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [204]:
#집합에 포함되는 단어들로 만들어진 문장을 정수 시퀀스로 만들어 보기
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded = tokenizer.texts_to_sequences([sub_text])[0]
encoded

[2, 5, 1, 6, 3, 7]

In [205]:
#to_categorical 원핫인코딩
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


#원-핫 인코딩의 한계

- 단어의 개수가 늘어날수록 벡터의 차원이 늘어남. 저장 공간 측면에서 비효율적.
- 유사도 표현 불가. 검색 시스템에서 문제됨.

-> 이를 해결하기 위해 잠재 의미를 반영한 다차원 공간 벡터화 가능. LSA, HAL, NNLM, RNNLM, Word2Vec, FastText, GloVe 등